In [1]:
import pandas as pd
import numpy as np
import time
from IPython.display import display
import matplotlib.pyplot as plt
from pandasql import sqldf
from sklearn import linear_model
import statsmodels.api as sm
from scipy import stats
import seaborn
from scipy.stats import pearsonr
from datetime import datetime, timedelta
import math

In [3]:
data = pd.read_csv("merged_final.csv")
data

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f3,ret_f4,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12
0,10026,198603.0,75.0,JJSF,12825,466032109,311812,30202030,0.201498,0.146460,...,-0.156250,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769
1,10026,198604.0,76.0,JJSF,12825,466032109,311812,30202030,0.985464,0.158181,...,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403
2,10026,198605.0,77.0,JJSF,12825,466032109,311812,30202030,0.715998,0.158346,...,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373
3,10026,198606.0,78.0,JJSF,12825,466032109,311812,30202030,0.672322,0.126541,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292
4,10026,198607.0,79.0,JJSF,12825,466032109,311812,30202030,0.638727,0.127120,...,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,0.114504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401583,93429,201802.0,458.0,CBOE,184500,12503M108,523210,40203040,0.516443,0.089356,...,-0.083817,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629
401584,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900
401585,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648
401586,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253
